In [ ]:
pip install tensorflow

In [3]:
import numpy as np
import os
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.preprocessing import image
import cv2

# Chargement des images
def load_images_from_folder(folder, label, target_size=(224, 224)):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, target_size)
            images.append(img)
            labels.append(label)
    return images, labels

# Charger les données
def load_data():
    # Chemins vers vos sous-dossiers
    pop_folder = "path_to_pop_folder"
    jazz_folder = "path_to_jazz_folder"
    death_metal_folder = "path_to_death_metal_folder"

    pop_images, pop_labels = load_images_from_folder(pop_folder, 0)  # 0 pour Pop
    jazz_images, jazz_labels = load_images_from_folder(jazz_folder, 1)  # 1 pour Jazz
    death_metal_images, death_metal_labels = load_images_from_folder(death_metal_folder, 2)  # 2 pour Death Metal

    # Combiner les données
    images = np.array(pop_images + jazz_images + death_metal_images)
    labels = np.array(pop_labels + jazz_labels + death_metal_labels)

    return images, labels

# Prétraitement des images : normalisation et vectorisation
def preprocess_images(images):
    # Normalisation : Diviser par 255 pour que les pixels soient dans [0, 1]
    images = images.astype('float32') / 255.0

    # Aplatir les images en vecteurs (224 x 224 x 3 -> 150528)
    images = images.reshape(images.shape[0], -1)
    
    # Application de PCA pour réduire la dimensionnalité (environ 500 dimensions pour un bon compromis)
    pca = PCA(n_components=500)
    images = pca.fit_transform(images)
    
    return images

# Définir et entraîner les modèles
def evaluate_models(X_train, X_test, y_train, y_test):
    models = {
        "KNN": KNeighborsClassifier(),
        "SVM": SVC(),
        "Logistic Regression": LogisticRegression(max_iter=1000)
    }

    for name, model in models.items():
        # Validation croisée
        print(f"Validation croisée pour {name} :")
        scores = cross_val_score(model, X_train, y_train, cv=5)
        print(f"Cross-validation accuracy: {scores.mean():.4f} ± {scores.std():.4f}")
        
        # Entraînement et évaluation du modèle
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        accuracy = accuracy_score(y_test, y_pred)
        print(f"{name} Accuracy: {accuracy:.4f}")
        print(f"Classification Report:\n{classification_report(y_test, y_pred)}")

# Main function pour exécuter le pipeline
def main():
    # Charger les données
    images, labels = load_data()
    
    # Diviser les données en training et test set
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
    
    # Prétraiter les images
    X_train = preprocess_images(X_train)
    X_test = preprocess_images(X_test)

    # Standardisation (important pour SVM et KNN)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Evaluer les modèles
    evaluate_models(X_train, X_test, y_train, y_test)

# Exécution du code
if __name__ == "__main__":
    main()


2025-01-02 17:08:33.214475: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-02 17:08:33.219224: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-02 17:08:33.234847: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735837713.256306   23728 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735837713.262446   23728 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-02 17:08:33.288280: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

FileNotFoundError: [Errno 2] No such file or directory: 'path_to_pop_folder'